 # 課題5: 評判分析

本課題ではAmazonに投稿された映画のレビュー(英語)を分析し、レビューがPositiveかNegativeかの判別を行います。

今回の課題のディレクトリには `Training_data` と `Test_data` が入っています。さらにその中には `pos` と `neg` というディレクトリがあり、Positiveなデータが `pos` に、Negativeなデータが `neg` に入っています。

- Training_data (positive用)、文章数 : 700
- Training_data (negative用)、文章数 : 700
- Test_data (positive用)、文章数 : 3
- Test_data (negative用)、文章数 : 3

（ ※学習用データ：1400、　テスト用データ：6、　合計 : 1406 の文章です）

`Training_data`を用いて機械学習を行い、その結果を元に、6つのTest dataがPositiveかNegativeかを判別してください。

レッスン9で学んだ内容を踏まえ、各セルに'#コメント'の内容を実行するコードを記入してください。

わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なモジュールのインポート

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 文章ファイル検索用
import glob
import collections
from sklearn.feature_extraction import DictVectorizer

# JupyterNotebook上でグラフを表示する設定
%matplotlib inline
# DataFrameで全ての列を表示する設定
pd.options.display.max_columns = None

## 2. データの読み込み

`./Amazon_review` 配下のフォルダからファイルを読み込みます。

In [3]:
# Training_dataフォルダ
train_neg_files = glob.glob("./Amazon_review/Training_data/neg/cv*")
train_pos_files = glob.glob("./Amazon_review/Training_data/pos/cv*")
# Test_dataフォルダ
test_neg_files = glob.glob("./Amazon_review/Test_data/neg/amazon*")
test_pos_files = glob.glob("./Amazon_review/Test_data/pos/amazon*")

In [4]:
# それぞれのファイル数を確認
print(len(train_neg_files))
print(len(train_pos_files))
print(len(test_neg_files))
print(len(test_pos_files))

700
700
3
3


In [5]:
# ファイル名をまとめたリスト filenames を用意
filenames = train_neg_files + train_pos_files + test_neg_files + test_pos_files
# filenamesの長さを確認
len(filenames)

1406

In [6]:
# エンコーディング用定数(以下で繰り返し指定するため定数にしておく)
ENCODING = 'utf-8'

In [7]:
# 最初のファイルの内容を確認
with open(filenames[0], 'r', encoding=ENCODING)as f:
    text = f.read()
    print(text)

united states , 1998 u . s . release date : 5/8/98 ( wide ) running length : 1 : 20 mpaa classification : r ( profanity , mature themes , sexual situations ) theatrical aspect ratio : 1 . 85 : 1 cast : jada pinkett smith , tommy davidson , dave chappelle , paula jai parker , ll cool j , darrel heath , michael ralph , duane martin director : daisy v . s . mayer producers : beth hubbard , michael hubbard screenplay : david c . johnson cinematography : jean lepine music : michel colombier u . s . distributor : new line cinema working in the motion picture industry must be a constant source of frustration for a front-line african american actress like jada pinkett smith . despite being one of the freshest talents available , pinkett smith has often been relegated to playing thankless supporting parts ( a low down dirty shame , the nutty professor ) . the problem is , of course , that there aren't many good roles available for black women . take away the likes of waiting to exhale , set it 

In [8]:
# Training_dataのposの最初のファイルの内容を確認
with open(train_pos_files[0], 'r', encoding=ENCODING)as f:
    text = f.read()
    print(text)

there are some works of art that are almost impossible to review , not because of their own complexity , but because of their legendary status which prevents the reviewer to say anything original . one of such masterpieces is casablanca , probably not the best film in the history of the seventh art , but definitely the most popular one . its popularity can be measured not in a multitude of more or less disguised remakes that were made in more than half a century since its premiere , but also in countless tributes and references that movie makers use in their works to this day . casablanca is also a movie that has the very rare virtue of both being praised by the critics and loved by general audience . one of the things that makes this film even more unique was the fact that it was doomed to fail , at least judging by conventional movie-making wisdom of its time . it was based on a broadway play so mediocre that it hadn't been produced on stage ; screenplay by three writers - julius g .

## 3. データの前処理

In [9]:
# 文字列の中で使われている単語ごとの数を返す
def get_word_count(text, min_length=3):
    # ノイズの除去：不要と思われる文字を除去する
    for ch in ".,:;!?-+*/=()[]{}<>~^#$@%&'\"_0123456789":
        text = text.replace(ch, ' ')

    # 形態素解析：文章を単語に分割
    _words = text.strip().split()

    # 表記のゆれの補正：
    # 単語のリストを受け取り、指定された文字数以上の単語だけを全て小文字にして返す
    _words = [_word.lower() for _word in _words if len(_word) >= min_length]

    # collections.Counterの戻り値は辞書型のサブクラス
    _count = collections.Counter(_words)

    # 辞書型に変換して返す
    return dict(_count)

In [10]:
# 最初のファイルでテスト
with open(filenames[0], 'r', encoding=ENCODING)as f:
    text = f.read()
    
get_word_count(text)

{'united': 1,
 'states': 1,
 'release': 1,
 'date': 2,
 'wide': 1,
 'running': 1,
 'length': 1,
 'mpaa': 1,
 'classification': 1,
 'profanity': 1,
 'mature': 1,
 'themes': 1,
 'sexual': 1,
 'situations': 2,
 'theatrical': 1,
 'aspect': 1,
 'ratio': 1,
 'cast': 1,
 'jada': 2,
 'pinkett': 6,
 'smith': 6,
 'tommy': 2,
 'davidson': 3,
 'dave': 1,
 'chappelle': 1,
 'paula': 1,
 'jai': 1,
 'parker': 1,
 'cool': 1,
 'darrel': 1,
 'heath': 1,
 'michael': 2,
 'ralph': 1,
 'duane': 1,
 'martin': 1,
 'director': 1,
 'daisy': 2,
 'mayer': 2,
 'producers': 1,
 'beth': 1,
 'hubbard': 2,
 'screenplay': 1,
 'david': 2,
 'johnson': 2,
 'cinematography': 1,
 'jean': 1,
 'lepine': 1,
 'music': 1,
 'michel': 1,
 'colombier': 1,
 'distributor': 1,
 'new': 1,
 'line': 2,
 'cinema': 1,
 'working': 1,
 'the': 33,
 'motion': 2,
 'picture': 1,
 'industry': 1,
 'must': 1,
 'constant': 2,
 'source': 1,
 'frustration': 1,
 'for': 8,
 'front': 1,
 'african': 1,
 'american': 1,
 'actress': 1,
 'like': 1,
 'despite':

In [11]:
# 単語ごとの数のリスト word_count_data を作成
word_count_data = []

for filename in filenames:
    with open(filename, 'r', encoding=ENCODING)as f:
        text = f.read()
        count = get_word_count(text)
        word_count_data.append(count)

In [12]:
# 単語ごとの数のリストの長さを確認
len(word_count_data)

1406

In [13]:
# リストの0番目を表示
word_count_data[0]

{'united': 1,
 'states': 1,
 'release': 1,
 'date': 2,
 'wide': 1,
 'running': 1,
 'length': 1,
 'mpaa': 1,
 'classification': 1,
 'profanity': 1,
 'mature': 1,
 'themes': 1,
 'sexual': 1,
 'situations': 2,
 'theatrical': 1,
 'aspect': 1,
 'ratio': 1,
 'cast': 1,
 'jada': 2,
 'pinkett': 6,
 'smith': 6,
 'tommy': 2,
 'davidson': 3,
 'dave': 1,
 'chappelle': 1,
 'paula': 1,
 'jai': 1,
 'parker': 1,
 'cool': 1,
 'darrel': 1,
 'heath': 1,
 'michael': 2,
 'ralph': 1,
 'duane': 1,
 'martin': 1,
 'director': 1,
 'daisy': 2,
 'mayer': 2,
 'producers': 1,
 'beth': 1,
 'hubbard': 2,
 'screenplay': 1,
 'david': 2,
 'johnson': 2,
 'cinematography': 1,
 'jean': 1,
 'lepine': 1,
 'music': 1,
 'michel': 1,
 'colombier': 1,
 'distributor': 1,
 'new': 1,
 'line': 2,
 'cinema': 1,
 'working': 1,
 'the': 33,
 'motion': 2,
 'picture': 1,
 'industry': 1,
 'must': 1,
 'constant': 2,
 'source': 1,
 'frustration': 1,
 'for': 8,
 'front': 1,
 'african': 1,
 'american': 1,
 'actress': 1,
 'like': 1,
 'despite':

### 行列への変換

In [14]:
# DictVectorizerを使用して行列に変換し、datasetに格納する
vec = DictVectorizer()
dataset = vec.fit_transform(word_count_data)

In [15]:
# datasetの大きさを確認
dataset

<1406x35075 sparse matrix of type '<class 'numpy.float64'>'
	with 445276 stored elements in Compressed Sparse Row format>

In [16]:
# 各列に対応した単語を取得
vec.get_feature_names()

['``the',
 '`accidentally',
 '`apollo',
 '`armageddon',
 '`at',
 '`atmosphere',
 '`batman',
 '`bats',
 '`believers',
 '`ben',
 '`bio',
 '`blade',
 '`blue',
 '`bonnie',
 '`boob',
 '`chef',
 '`city',
 '`closure',
 '`cooks',
 '`cops',
 '`cruel',
 '`cute',
 '`dad',
 '`dammnit',
 '`damn',
 '`dark',
 '`darlene',
 '`dawson',
 '`daylight',
 '`diddly',
 '`disclaiming',
 '`down',
 '`dragonheart',
 '`drive',
 '`dumb',
 '`d駛a',
 '`eat',
 '`elvis',
 '`empire',
 '`enough',
 '`ev',
 '`ex',
 '`experience',
 '`father',
 '`fatty',
 '`fiji',
 '`frank',
 '`friends',
 '`gangsta',
 '`georgia',
 '`give',
 '`go',
 '`hang',
 '`hanging',
 '`hello',
 '`homage',
 '`horror',
 '`hot',
 '`how',
 '`if',
 '`inspector',
 '`it',
 '`jaws',
 '`jedi',
 '`jeez',
 '`just',
 '`kids',
 '`kirk',
 '`kyle',
 '`last',
 '`life',
 '`like',
 '`long',
 '`loose',
 '`macho',
 '`marketing',
 '`mission',
 '`not',
 '`oh',
 '`one',
 '`padawan',
 '`parental',
 '`political',
 '`power',
 '`praise',
 '`pump',
 '`ready',
 '`return',
 '`ronna',
 

## 4. 機械学習の実施

In [17]:
# 必要なライブラリのインポート
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [18]:
# 目的変数 Y の用意
# neg700 + pos700 + neg3 + pos3 = 1406
Y = [0]*700 + [1]*700 + [0]*3 + [1]*3

In [21]:
# データの分割
# dataset の先頭から1400を X_train に、6を X_test に代入
# Y       の先頭から1400を Y_train に、6を Y_test に代入
X_train, X_test = dataset[:1400], dataset[1400:]
Y_train, Y_test = Y[:1400], Y[1400:]
# X_trainとY_trainを、train_test_splitを使って7:3に分割
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)


In [22]:
# ロジスティック回帰モデルの作成
logistic_model = LogisticRegression(solver='lbfgs')
# モデルの学習
logistic_model.fit(X_train, Y_train)
# 検証データによる予測
Y_pred = logistic_model.predict(X_valid)

# 検証データによるモデルの評価
print(classification_report(Y_valid, Y_pred))

              precision    recall  f1-score   support

           0       0.78      0.78      0.78       207
           1       0.79      0.79      0.79       213

    accuracy                           0.79       420
   macro avg       0.79      0.79      0.79       420
weighted avg       0.79      0.79      0.79       420



/home/ec2-user/.pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## 5. テストデータによる評価

最後にテストデータによる評価を行いましょう

In [23]:
# テストデータによる予測
Y_pred = logistic_model.predict(X_test)

# モデルの評価
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67         3
           1       0.67      0.67      0.67         3

    accuracy                           0.67         6
   macro avg       0.67      0.67      0.67         6
weighted avg       0.67      0.67      0.67         6

